## Script to generate a VCF from the two outgroups
#### The file generated with the script GENOTYPES_outgroup_concatenate.py has to be used.
#### This file gives information on the genotypes of the two individuals SRR7440094 and SRR7440095 for each locus

### Get positions of the filtered SNPs from the file with the 173 samples (WGS dataset):

In [34]:
dico_positions_SNPs_chr1={}
with open("list_positions_SNPs_chr1.txt", "r") as file_snps:
    for snp in file_snps:
        snp_split=snp.split("\t")
        position=snp_split[0]
        allele_ref=snp_split[1]
        allele_alt=snp_split[2][:-1]
        dico_positions_SNPs_from_173_samples[position]={}
        dico_positions_SNPs_from_173_samples[position]["allele_ref"]=allele_ref
        dico_positions_SNPs_from_173_samples[position]["alele_alt"]=allele_alt
print(len(dico_positions_SNPs_from_173_samples))

3572843


In [35]:
dico_genotypes={}
#the file GENOTYPES_SRR7440094-5_CONCATENATED_chr1.txt is obtained from the script GENOTYPES_outgroup_concatenate.py
with open("GENOTYPES_SRR7440094-5_CONCATENATED_chr1.txt", "r") as gen:
    for g in gen:
        gen_split=g.split("\t")
        position=gen_split[0]
        if str(position) != "position":
            dico_genotypes[position]={}
            ref=gen_split[1]
            o1=gen_split[2]
            o2=gen_split[5]
            dp1=gen_split[3]
            dp2=gen_split[6]
            dico_genotypes[position]["outgroup1"]=o1
            dico_genotypes[position]["outgroup2"]=o2
            dico_genotypes[position]["DP1"]=dp1
            dico_genotypes[position]["DP2"]=dp2
            dico_genotypes[position]["REF"]=ref


print(len(dico_genotypes))

24636455
30935
{'outgroup1': 'TT', 'outgroup2': '.', 'DP1': '1', 'DP2': '.', 'REF': 'T'}


### Write the new VCF:

In [5]:
print(len(dico_positions_SNPs_chr1))
print(len(dico_genotypes))

4654641
28963846


In [36]:
pos_problem=[]
chromosome="chr1"
with open("vcf_chr1_Ped_outgroup.vcf", "w") as vcf_to_generate:
    for position in dico_positions_SNPs_from_173_samples: # {'allele_ref': 'T', 'alele_alt': 'C'}
        if (str(position[0]) != "p"):
            if str(position) in dico_genotypes: # {'outgroup1': '.', 'outgroup2': 'GG', 'DP1': '.', 'DP2': '1'}
                REF=dico_genotypes[position]["REF"]
                outgroup1=dico_genotypes[position]["outgroup1"] ##ex: AA
                DP1=dico_genotypes[position]["DP1"]
                outgroup2=dico_genotypes[position]["outgroup2"] ##ex: TT
                DP2=dico_genotypes[position]["DP2"]
                ref_allele_vcf_173=dico_positions_SNPs_from_173_samples[position]["allele_ref"]
                alt_allele_vcf_173=dico_positions_SNPs_from_173_samples[position]["alele_alt"]
                if str(REF)!=str(ref_allele_vcf_173):
                    pos_problem.append(position)
                ### OUTGROUP2 only:
                if str(outgroup1)=="." and str(outgroup2)!=".": #outgroup1 missing data but not outgroup 2
                    if int(DP2)>=2:
                        allele1_outgroup2=outgroup2[0]
                        allele2_outgroup2=outgroup2[1]
                        # if outgroup homozygote genotype:
                        ## REF: 0/0
                        if str(allele1_outgroup2)==str(ref_allele_vcf_173) and str(allele2_outgroup2)==str(ref_allele_vcf_173):
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP="+str(DP2)+";ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t0/0:0,0:"+str(DP2)+":20:0,0,0\n")
                        ## ALT: 1/1
                        if str(allele1_outgroup2)==str(alt_allele_vcf_173) and str(allele2_outgroup2)==str(alt_allele_vcf_173):
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP="+str(DP2)+";ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t1/1:0,0:"+str(DP2)+":20:0,0,0\n")
                        # if outgroup != REF or ALT or hetero => missing data
                        if (str(allele1_outgroup2)!=str(allele2_outgroup2)) or (str(allele1_outgroup2)!=str(ref_allele_vcf_173) and str(allele2_outgroup2)!=str(ref_allele_vcf_173) and str(allele1_outgroup2)!=str(alt_allele_vcf_173) and str(allele2_outgroup2)!=str(alt_allele_vcf_173)):
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")
                        # if hetero: => missing data
                        #if str(allele1_outgroup2)!=str(allele2_outgroup2) and :
                        #    vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n") 
                    if  int(DP2)<2: # outgroup => missing data
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")                

                ### OUTGROUP1 only:
                if str(outgroup2)=="." and str(outgroup1)!=".":
                    if int(DP1)>=2:
                        allele1_outgroup1=outgroup1[0]
                        allele2_outgroup1=outgroup1[1]    
                        # if outgroup homozygote genotype:
                        ## REF: 0/0
                        if str(allele1_outgroup1)==str(ref_allele_vcf_173) and str(allele2_outgroup1)==str(ref_allele_vcf_173):
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP="+str(DP1)+";ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t0/0:0,0:"+str(DP1)+":20:0,0,0\n")
                        ## ALT: 1/1
                        if str(allele1_outgroup1)==str(alt_allele_vcf_173) and str(allele2_outgroup1)==str(alt_allele_vcf_173):
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP="+str(DP1)+";ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t1/1:0,0:"+str(DP1)+":20:0,0,0\n")
                        # if outgroup != REF or ALT or hetero
                        if (str(allele1_outgroup1)!=str(allele2_outgroup1)) or (str(allele1_outgroup1)!=str(ref_allele_vcf_173) and str(allele2_outgroup1)!=str(ref_allele_vcf_173) and str(allele1_outgroup1)!=str(alt_allele_vcf_173) and str(allele2_outgroup1)!=str(alt_allele_vcf_173)):
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")
                        # if hetero:
                        #if str(allele1_outgroup1)!=str(allele2_outgroup1):
                        #    vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")                
                    if int(DP1)<2:
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")                

                ### TWO OUTGROUPS:            
                if str(outgroup1)!="." and str(outgroup2)!=".":                        
                    # if the two outgroups are the same:
                    if str(outgroup1)==str(outgroup2):
                        DP_sum=int(DP1)+int(DP2)                            
                        if int(DP_sum)>=2:
                            allele1_outgroup=outgroup1[0]
                            allele2_outgroup=outgroup1[1]                                
                            # if outgroup homozygote genotype:
                            ## REF: outgroup 0/0
                            if (str(allele1_outgroup)==str(allele2_outgroup)) and (allele1_outgroup)==str(ref_allele_vcf_173) :
                                vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP="+str(DP_sum)+";ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t0/0:0,0:"+str(DP_sum)+":0:0,0,0\n")                                                
                            # if outgroup homozygote genotype:
                            ## ALT: outgroup 1/1
                            if (str(allele1_outgroup)==str(allele2_outgroup)) and (allele1_outgroup)==str(alt_allele_vcf_173) :
                                vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP="+str(DP_sum)+";ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t1/1:0,0:"+str(DP_sum)+":0:0,0,0\n")                                                
                            # if hetero:
                            #if str(allele1_outgroup)!=str(allele2_outgroup):
                            #    vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")                                           
                            # if outgroup != REF or ALT or hetero => missing data
                            if (str(allele1_outgroup)!=str(allele2_outgroup)) or ( (str(allele1_outgroup)==str(allele2_outgroup)) and str(allele1_outgroup)!=str(ref_allele_vcf_173) and str(allele1_outgroup)!=str(alt_allele_vcf_173) ):
                                vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")
                        if int(DP_sum)<2:
                            vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")                                        
                    # if the two outgroups are different:
                    if str(outgroup1)!=str(outgroup2):
                        vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")                
            if str(position) not in dico_genotypes: # outgroup with missing data
                ref_allele_vcf_173=dico_positions_SNPs_from_173_samples[position]["allele_ref"]
                alt_allele_vcf_173=dico_positions_SNPs_from_173_samples[position]["alele_alt"]
                vcf_to_generate.write(str(chromosome)+"\t"+str(position)+"\t.\t"+str(ref_allele_vcf_173)+"\t"+str(alt_allele_vcf_173)+"\t1.0\tPASS\tAC=1;AF=0.0;AN=0;BaseQRankSum=0;DP=0;ExcessHet=0;FS=0;InbreedingCoeff=0;MLEAC=0;MLEAF=0;MQ=0;MQRankSum=0;QD=0;ReadPosRankSum=0;SOR=0\tGT:AD:DP:GQ:PL\t./.:0,0:0:0:0,0,0\n")
